In [2]:
import pandas as pd

train_df = pd.read_csv("train_processed.csv")

train_df.head()


,unit,time,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL,failure_24h
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191.0,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190.0,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189.0,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188.0,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187.0,0


In [3]:
sensor_cols = [col for col in train_df.columns if col.startswith("sensor_")]
sensor_cols

['sensor_1',
 'sensor_2',
 'sensor_3',
 'sensor_4',
 'sensor_5',
 'sensor_6',
 'sensor_7',
 'sensor_8',
 'sensor_9',
 'sensor_10',
 'sensor_11',
 'sensor_12',
 'sensor_13',
 'sensor_14',
 'sensor_15',
 'sensor_16',
 'sensor_17',
 'sensor_18',
 'sensor_19',
 'sensor_20',
 'sensor_21']

In [4]:
train_df = train_df.sort_values(by=["unit", "time"])

In [5]:
windows = [1, 6, 12]

for window in windows:
    for sensor in sensor_cols:
        train_df[f"{sensor}_roll_mean_{window}"] = (
            train_df
            .groupby("unit")[sensor]
            .rolling(window)
            .mean()
            .reset_index(level=0, drop=True)
        )

In [6]:
for window in windows:
    for sensor in sensor_cols:
        train_df[f"{sensor}_roll_std_{window}"] = (
            train_df
            .groupby("unit")[sensor]
            .rolling(window)
            .std()
            .reset_index(level=0, drop=True)
        )

/tmp/ipython-input-3733395937.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f"{sensor}_roll_std_{window}"] = (
/tmp/ipython-input-3733395937.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f"{sensor}_roll_std_{window}"] = (
/tmp/ipython-input-3733395937.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,

In [7]:
lags = [1, 2]

for lag in lags:
    for sensor in sensor_cols:
        train_df[f"{sensor}_lag_{lag}"] = (
            train_df
            .groupby("unit")[sensor]
            .shift(lag)
        )

/tmp/ipython-input-333844504.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f"{sensor}_lag_{lag}"] = (
/tmp/ipython-input-333844504.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[f"{sensor}_lag_{lag}"] = (
/tmp/ipython-input-333844504.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fr

In [8]:
# Count NaNs per row
nan_counts = train_df.isna().sum(axis=1)

nan_counts.describe()

,0
count,20631.000000
mean,24.562600
std,16.480793
min,21.000000
25%,21.000000
50%,21.000000
75%,21.000000
max,147.000000


In [10]:
train_df = train_df.groupby("unit").apply(
    lambda x: x.iloc[12:]
).reset_index(drop=True)

/tmp/ipython-input-1640393437.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby("unit").apply(


In [11]:
train_df.shape

(19431, 196)

In [12]:
train_df.to_csv("train_features.csv", index=False)
